In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-2"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp

# create different test functions as tensorflow functions

In [3]:
def testfunc_sin(x, a, b, c):
    a=tf.cast(a,tf.float64)
    b=tf.cast(b,tf.float64)
    c=tf.cast(c,tf.float64)
    return a*tf.sin(b*x)+c

In [4]:
def testfunc_lin(x, a, b):
    a=tf.cast(a,tf.float64)
    b=tf.cast(b,tf.float64)
    return a*x+b

In [5]:
def testfunc_exp(x, a, b):
    a=tf.cast(a,tf.float64)
    b=tf.cast(b,tf.float64)
    return tf.exp(a*x)+b

# create smeared test data 

In [6]:
x_vals = np.array(np.linspace(0,2*np.pi,50))

In [7]:
y_vals_sin = []
y_vals_lin = []
y_vals_exp = []

In [8]:
for el in x_vals:
    y_vals_sin.append(testfunc_sin(el, np.random.normal(0.5,0.1), np.random.normal(0.5,0.1), np.random.normal(0.5,0.1)))
    y_vals_lin.append(testfunc_lin(el, np.random.normal(0.5,0.1), np.random.normal(0.5,0.1)))
    y_vals_exp.append(testfunc_exp(el, np.random.normal(0.5,0.1), np.random.normal(0.5,0.1)))

# it can be used any loss function with f(y_pred, y_true) which returns a scalar

In [9]:
def loss_func(y_pred, y_true):
    return tf.reduce_mean((y_pred - y_true )**2)


# create test function with weights

In [10]:
def loss_func_w(y_pred, y_true, weights):
    return tf.reduce_mean((y_pred - y_true )**2)+tf.reduce_mean(weights)
 

In [11]:
weights = np.ones(x_vals.shape[0])

# import Fitter and test it for the different functions and normal fitting + BS fitting

In [12]:
from Fitter import curve_fit, curve_fit_BS, curve_fit_raw

In [13]:
testfunclist = [testfunc_sin, testfunc_lin, testfunc_exp]
y_vals_list = [y_vals_sin, y_vals_lin, y_vals_exp]
namelist = ["sin","lin","exp"]

In [14]:
for el in zip(testfunclist,y_vals_list,namelist):
    a,b,c = curve_fit_raw(x_vals, np.array(el[1]), el[0], loss=loss_func)
    a,b,c = curve_fit_raw(x_vals, np.array(el[1]), el[0], loss=loss_func_w, weights = weights)
    a,c = curve_fit(x_vals, np.array(el[1]), el[0], loss=loss_func)
    print()
    print()
    print("function: "+el[2])
    print("parameters:")
    print(a)
    print("covariance matrix:")
    print(c)
    a,c = curve_fit(x_vals, np.array(el[1]), el[0], loss=loss_func_w, weights = weights)
    a,c = curve_fit_BS(x_vals, np.array(el[1]), el[0], loss=loss_func)
    print("parameters BS:")
    print(a)
    print("errors BS:")
    print(c)
    
    a,c = curve_fit_BS(x_vals, np.array(el[1]), el[0], loss=loss_func_w, weights = weights)




function: sin
parameters:
[0.38601232, 0.46180573, 0.5685764]
covariance matrix:
[[ 0.32446882 -0.02246676 -0.2254054 ]
 [-0.02246675  0.03450719  0.02831388]
 [-0.22540537  0.02831388  0.18729536]]
parameters BS:
[0.3266318 0.6691086 0.6260999]
errors BS:
[0.14696522 0.44122157 0.10335104]


function: lin
parameters:
[0.5023073, 0.54026407]
covariance matrix:
[[ 0.03671923 -0.11535686]
 [-0.11535686  0.48813632]]
parameters BS:
[0.49736375 0.56711304]
errors BS:
[0.02323407 0.06713528]


function: exp
parameters:
[0.5307627, -0.21283449]
covariance matrix:
[[ 1.0674380e-02 -4.1242674e-01]
 [-4.1242674e-01  4.1157516e+01]]
parameters BS:
[ 0.5194047 -0.2677133]
errors BS:
[0.01023802 0.16328071]


# initial parameters should be provided as tf.Variable

In [15]:
initial_parameters = [tf.Variable(1.),tf.Variable(1.),tf.Variable(1.)]

In [16]:
a,c = curve_fit(x_vals, np.array(y_vals_sin), testfunc_sin, initial_parameters=initial_parameters)

# use Keras loss

In [17]:
lhuber = tf.keras.losses.Huber()

In [18]:
a,c = curve_fit(x_vals, np.array(y_vals_sin), testfunc_sin, loss = lhuber, initial_parameters=initial_parameters)

In [19]:
a

[0.38623515, 0.46178913, 0.568419]

# test many parameters

In [20]:
n_tests = 5
y_lin_list = []
y_sin_list = []
y_exp_list = []
for i in range(n_tests):
    y_lin_list.append([])
    y_sin_list.append([])
    y_exp_list.append([])

In [21]:
a_lin = np.random.uniform(size = n_tests)
b_lin = np.random.uniform(size = n_tests)
sigma_lin = np.random.uniform(size = n_tests)

a_exp = np.random.uniform(size = n_tests)
b_exp = np.random.uniform(size = n_tests)
sigma_exp = np.random.uniform(size = n_tests)

a_sin = np.random.uniform(size = n_tests)
b_sin = np.random.uniform(size = n_tests)
c_sin = np.random.uniform(size = n_tests)
sigma_sin = np.random.uniform(size = n_tests)

In [22]:
for idx,el in enumerate(zip(y_sin_list, y_lin_list, y_exp_list)):
        for x in x_vals:
            el[0].append(testfunc_sin(x, a_sin[idx], b_sin[idx], c_sin[idx])+np.random.normal(0,sigma_sin[idx]))
            el[1].append(testfunc_lin(x, a_lin[idx], b_lin[idx])+np.random.normal(0,sigma_lin[idx]))
            el[2].append(testfunc_exp(x, a_exp[idx], b_exp[idx])+np.random.normal(0,sigma_exp[idx]))

In [ ]:
pars_out_sin = []
pars_out_lin = []
pars_out_exp = []
errs_out_sin = []
errs_out_lin = []
errs_out_exp = []
pars_out_sinBS = []
pars_out_linBS = []
pars_out_expBS = []
errs_out_sinBS = []
errs_out_linBS = []
errs_out_expBS = []

for i in range(n_tests):
        print(i)
        #a,b = curve_fit(x_vals, np.array(y_sin_list[i]), testfunc_sin)
        #pars_out_sin.append(a)
        #errs_out_sin.append(np.sqrt(np.diag(b)))
        #a,b = curve_fit_BS(x_vals, np.array(y_sin_list[i]), testfunc_sin)
        #pars_out_sinBS.append(a)
        #errs_out_sinBS.append(b)
        a,b = curve_fit(x_vals, np.array(y_lin_list[i]), testfunc_lin)
        pars_out_lin.append(a)
        errs_out_lin.append(np.sqrt(np.diag(b)))
        a,b = curve_fit_BS(x_vals, np.array(y_lin_list[i]), testfunc_lin)
        pars_out_linBS.append(a)
        errs_out_linBS.append(b)
        #a,b = curve_fit(x_vals, np.array(y_exp_list[i]), testfunc_exp)
        #pars_out_exp.append(a)
        #errs_out_exp.append(np.sqrt(np.diag(b)))
        #a,b = curve_fit_BS(x_vals, np.array(y_exp_list[i]), testfunc_exp)
        #pars_out_expBS.append(a)
        #errs_out_expBS.append(b)


0


In [ ]:
#np.array(pars_out_sin)[:,0]

dictsin = {"a": a_sin, "b": b_sin, "c": c_sin, "a_pred": np.array(pars_out_sin)[:,0], "b_pred": np.array(pars_out_sin)[:,1], "c_pred": np.array(pars_out_sin)[:,2],
          "a_err": np.array(errs_out_sin)[:,0], "b_err": np.array(errs_out_sin)[:,1], "c_err": np.array(errs_out_sin)[:,2],
          "a_predBS": np.array(pars_out_sinBS)[:,0], "b_predBS": np.array(pars_out_sinBS)[:,1], "c_predBS": np.array(pars_out_sinBS)[:,2],
          "a_errBS": np.array(errs_out_sinBS)[:,0], "b_errBS": np.array(errs_out_sinBS)[:,1], "c_errBS": np.array(errs_out_sinBS)[:,2]
          }

In [ ]:
dictlin = {"a": a_lin, "b": b_lin, "a_pred": np.array(pars_out_lin)[:,0], "b_pred": np.array(pars_out_lin)[:,1],
          "a_err": np.array(errs_out_lin)[:,0], "b_err": np.array(errs_out_lin)[:,1],
          "a_predBS": np.array(pars_out_linBS)[:,0], "b_predBS": np.array(pars_out_linBS)[:,1],
          "a_errBS": np.array(errs_out_linBS)[:,0], "b_errBS": np.array(errs_out_linBS)[:,1]
          }

dictexp = {"a": a_exp, "b": b_exp, "a_pred": np.array(pars_out_exp)[:,0], "b_pred": np.array(pars_out_exp)[:,1],
          "a_err": np.array(errs_out_exp)[:,0], "b_err": np.array(errs_out_exp)[:,1],
          "a_predBS": np.array(pars_out_expBS)[:,0], "b_predBS": np.array(pars_out_expBS)[:,1],
          "a_errBS": np.array(errs_out_expBS)[:,0], "b_errBS": np.array(errs_out_expBS)[:,1]
          }

In [ ]:
#dfsin = pd.DataFrame(dictsin)
dflin = pd.DataFrame(dictlin)
#dfexp = pd.DataFrame(dictexp)

In [ ]:
dflin.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist((dflin["a"]-dflin["a_pred"])/dflin["a_err"], bins=50)

In [ ]:
np.mean((dflin["a"]-dflin["a_pred"])/dflin["a_err"])

In [ ]:
np.std((dflin["a"]-dflin["a_pred"])/dflin["a_err"])